In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

mpdf = pd.read_excel("/home/rupnik/parlamint/BiH/BiH_MPs_1998-2022_v1.xlsx")
# mpdf["reference_name"] = mpdf.fullname.apply(
#     lambda fullname: "".join(fullname.split()).replace(",", "").replace(" ", "").replace("–", "-")
#     )
partiesdf = pd.read_excel("/home/rupnik/parlamint/BiH/BiH_electoral_lists_1998-2022_v1.xlsx")

In [2]:
from pathlib import Path
from tempfile import NamedTemporaryFile
from utils import prepare_interim_files, construct_TEI
from parse import compile
from concurrent.futures import ProcessPoolExecutor
import shutil

datadir = Path("/home/rupnik/parlamint/BiH/S_data/")
files = list(datadir.glob("*"))

for f in files:
    if not "sjednica" in f.name:
        shutil.move(
            str(datadir/f.name), 
            str(datadir/ (f.name).
                            replace("sjdenica", "sjednica").
                            replace("sjedica", "sjednica").
                            replace("sjedinca", "sjednica")
                              )
        )

In [3]:

# Identify all (T,S) pairs:
files = list(datadir.glob("*"))
pattern = "{prefix}_T{term:02}_S{session}. {suffix}.{ending}"
p = compile(pattern)
terms_and_sessions_and_suffices = set(
    (p.parse(i.name).named["term"], p.parse(i.name).named["session"], p.parse(i.name).named["suffix"]) 
    for i in files)


def process(term, session, suffix) -> None:
    term = int(term)
    text_path = datadir / (f"text_T{term:02}_S{session}. {suffix}.txt")
    meta_path = datadir / (f"meta_T{term:02}_S{session}. {suffix}.tsv")
    mp_path = Path("/home/rupnik/parlamint/BiH") / Path("BiH_MPs_1998-2022_v1.xlsx")
    parties_path = Path("/home/rupnik/parlamint/BiH") / Path("BiH_electoral_lists_1998-2022_v1.xlsx")
    
    assert text_path.exists(), "No text!"
    merged_file = NamedTemporaryFile()
    prepare_interim_files(
        text_path = text_path,
        meta_path = meta_path,
        mp_path = mp_path,
        parties_path= parties_path,
        out_file = merged_file.name
    )
    session_appended = session + suffix.replace("sjednica", "").\
        replace("hitna", "h").replace("nastavak2", "n2").replace("nastavak3", "n3").\
        replace("nastavak", "n").replace("vanredna", "v").replace("konst", "k").\
        replace("nastavat", "n").replace("nastavk", "n")
    session_appended = "".join(session_appended.split())
    construct_TEI(
        pickled_file = merged_file.name,
        session_index=session,
        term_index=term,
        data_language_code = "bs",
        out_file = Path("/home/rupnik/parlamint/BiH/S/") / f"ParlaMint-BA_T{term:02}S{session_appended}.xml"
        )
    
    
# with ProcessPoolExecutor(max_workers=25) as executor:
#     futures = executor.map(process, 
#                           [term for term, session, suffix in terms_and_sessions_and_suffices], 
#                           [session for term, session, suffix in terms_and_sessions_and_suffices],
#                           [suffix for term, session, suffix in terms_and_sessions_and_suffices]
# )

for term, session, suffix in list(terms_and_sessions_and_suffices)[:]:
    process(term, session, suffix)

  0%|          | 0/199 [00:00<?, ?it/s]2022-11-16 16:24:01 INFO: Loading these models for language: hr (Croatian):
| Processor | Package  |
------------------------
| tokenize  | standard |

2022-11-16 16:24:01 INFO: Use device: cpu
2022-11-16 16:24:01 INFO: Loading: tokenize
2022-11-16 16:24:01 INFO: Done loading processors!
100%|██████████| 189/189 [00:01<00:00, 143.43it/s]


Getting errors for  nan nan nan


100%|██████████| 370/370 [00:02<00:00, 129.86it/s]
